# Investigating Blocked or Slow Finalizers

In this case study, we will walk through how to debug finalizers that are slow, long running, or blocked.  This is sometimes the root cause of memory leaks or excess memory usage in a C# application, so it's important to understand how to debug when this is happening.

## Finalizers and the finalizer queue

The "Finalizer Queue" in the .Net Runtime processes finalizable objects after the GC has identified that the object is no longer reachable.  This is done on a deticated thread called the "Finalizer Thread".  There is only ever one finalizer thread processing finalizable objects.  This means that if a process ever creates finalizable objects faster than they can be processed by the FinalizerQueue, your process will eventually run out of memory.  This isn't a "memory leak" in the traditional sense, as the .Net Runtime knows where all of that memory is.  It's trying to clean up that memory as quickly as possible, but isn't able to keep up with demand.

As a refresher, finalizers are defined in C# code as follows:

``` C#
class Foo
{
    ~Foo()
    {
        // This is a finalizer, it will be executed on a finalizer thread.
    }
}
```

## In the real world

In this case study, we will be taking a look at a finalizer that is fully blocked.  We've caused a deadlock by taking a monitor and never releasing it.  This is the easiest way to make sure our scenario is easy to debug and understand.  Keep in mind that even just having *slow* code in a finalizer can lead to a problem if too many objects of that type are generated.

For example, we have seen issues where finalizers caused excess memory usage in the process due to:

1. Blocked threads, such as a finalizer trying to take a lock held by another thread.
2. Async operations like a network operation, or file IO that takes a long time to complete.
3. Attempting to call into an STA apartment (like the UI thread) from a finalizer.
4. Extensive retry code (especially for network operations).
5. Or just any kind of long-running operation that is occurring on the finalizer thread.

## Practical advice for avoiding these problems

Where possible, these kinds of problems can be avoided using the `Dispose()` pattern.  Classes with finalizers should implement `IDisposable` and be used with a `using` statement to clean up any state.  This is the canonical example of a disposable/finalizable class:

```C#
    public class DisposableResource : IDisposable
    {
        // Flag to detect redundant calls
        private bool _disposed = false;

        // Public method to use the resource
        public void UseResource()
        {
            if (_disposed)
                throw new ObjectDisposedException("DisposableResource");

            Console.WriteLine("Using the resource.");
        }

        // Implement IDisposable
        public void Dispose()
        {
            Dispose(true);
            // Be sure to suppress finalization:
            GC.SuppressFinalize(this);
        }

        // Protected implementation of Dispose pattern
        protected virtual void Dispose(bool disposing)
        {
            if (!_disposed)
            {
                if (disposing)
                {
                    // Clean up managed objects.  That means if this class holds other IDiposable objects,
                    // call their Dispose() methods here.  Note that "cleaning up" does not mean assigning
                    // it's fields to null, which I often see.  You can do that if you want, or have a
                    // reason to do so, but we mean calling Dispose on other objects in this block.
                }

                // Clean up native objects and memory.
                // For example, calling out to release native resources.

                _disposed = true;
            }
        }

        // Finalizer
        ~DisposableResource()
        {
            // Do not change this code. Put cleanup code in Dispose(bool disposing) above.
            Dispose(false);
        }
    }
```

Implementing this pattern ensures that we efficiently clean up everything when `Dispose` is called on the thread using the object.  This means the finalizer thread will never have to process it, avoiding the problem!

Of course, some libraries you use may not properly implement this pattern, sometimes what you are implementing doesn't neatly translate to that pattern, and sometimes it's simply not possible to call `Dispose` in certain circumstances even when we know it would be better to do so.  The rest of this document explains how we go about debugging and investigating something like this goes wrong.

## Setting up our enviroment

Before we dive into this case study, the following code cells will load up our environment and the target crash dump.  If you are running this yourself, be sure to build the repro with `dotnet build` and set your install path of WinDbg after `#!dbgengPath` below.  Otherwise, skip ahead to the next section.

In [1]:
// Boilerplate to get started: Load up the WinDbgKernel

#r "..\WinDbgKernel\bin\Debug\net8.0-windows10.0.17763.0\publish\WinDbgKernel.dll"
#!import "..\WinDbgKernel\extension.dib"

Installed Packages Microsoft.Diagnostics.Runtime, 3.1.512801

In [2]:
#!dbgengPath d:\amd64
#!loadDump ..\BlockedFinalizer\bin\release\net8.0\BlockedFinalizer.dmp


Dump file '..\BlockedFinalizer\bin\release\net8.0\BlockedFinalizer.dmp' loaded successfully.


## Our BlockedFinalizer repro

Take a brief look at our [BlockedFinalizer.cs](../BlockedFinalizer/BlockedFinalizer.cs) repro.  We are setting up a synthetic situation where our finalizer will attempt to take a lock that is not available:

```C#
    ~BlockedFinalizer()
    {
        ...

        // Held by another thread which never releases it.
        lock (Sync)
        {
            Console.WriteLine("Never executes, lock(Sync) blocks forever.");
        }
    }
```

This completely halts the finalizer thread, and we will now never make progress in finalizing any object.  Any objects which the GC collects and are determined ready for finalization will now never disappear from the heap.  It also means that all of the objects that are only referenced by these "dead" finalizable objects will also not be collected.  For example, in [BlockedFinalizer.cs](../BlockedFinalizer/BlockedFinalizer.cs), the list and objects contained in `BlockedFinalizer._list` cannot ever be swept by the GC.  At least, not until the finalizer for each `BlockedFinalizer` completes (which it never will in this example).

Of course, this is a synthetic and simple example as it's very rare to simply deadlock on the Finalizer thread.  However, this is similar enough to many real root causes of failures that we have investigated in the past that it will serve as a good example of how to debug this scenario.

## The !FinalizeQueue command

SOS offers a `!FinalizeQueue` command to inspect finalizable objects.  The standard `!FinalizeQueue` command will search the entire GC heap for objects that are finalizable.  The base command does *not* list objects that are "ready" for finalization, only that these objects will eventually be finalized when they are no longer referenced if `GC.SuppressFinalize` is not called on them.  By default, this command will list out every individual object, so we will just ask for a summary of objects with the `-stat` parameter.

As you can see below, we have 999 `BlockedFinalizer` objects that it found to be finalizable.  We allocated 1000, but one of them is already being processed by the finalizer thread, so it's actually no longer a 'finalizable' object by the time we inspect the heap here.  We also see several other objects that are used by the .Net Runtime and its standard library for various purposes.  This is all normal.

Remember, again, that this is a mix of both 'live' and 'dead' objects that are currently on the heap.  The 'dead' finalizable objects will only be cleaned up when a GC is performed and locates them, then their finalizer method is called by the finalizer thread.  (There are other caveats here, like resurrection that are beyond the scope of this document.)

Here is that full list of all finalizable objects living on the heap:

In [3]:
* This is a list of all finalizable objects on the heap.
* This includes ones that are live and not yet ready to be finalized.
!FinalizeQueue -stat

0:000> !FinalizeQueue -stat
SyncBlocks to be cleaned up: 0
Free-Threaded Interfaces to be released: 0
MTA Interfaces to be released: 0
STA Interfaces to be released: 0
----------------------------------

Heap 0
generation 0 has 16 objects (18e7206e410->18e7206e490)
generation 1 has 0 objects (18e7206e410->18e7206e410)
generation 2 has 0 objects (18e7206e410->18e7206e410)
Ready for finalization 1,011 objects (18e7206e590->18e72070528)
------------------------------
Statistics for all finalizable objects (including all objects ready for finalization):
Statistics:
          MT Count TotalSize Class Name
7ff816d35e88     1        40 System.Gen2GcCallback
7ff816d377a8     2        48 System.Threading.ThreadInt64PersistentCounter+ThreadLocalNodeFinalizationHelper
7ff816bb5710     4        96 System.WeakReference<System.Diagnostics.Tracing.EventSource>
7ff816cfc400     3        96 Microsoft.Win32.SafeHandles.SafeWaitHandle
7ff816c06970     4       128 Internal.Win32.SafeHandles.SafeRegistryHa

## What objects are 'ready' for finalization?

We can find what objects are 'ready' for finalization by using `!FinalizeQueue` with the `-allReady` parameter.  This narrows our output down to a handful of objects that *may* be processed by the FinalizerQueue the next time a GC completes and the finalizer thread is woken.

Extra nitpicky details:  The caveat here is that we may perform a Gen0 GC (only collecting generation 0 objects and not the whole heap), which means we may not sweep some dead finalizable objects in every single GC.  There may also be implementation details of the GC about when the finalizer thread is woken which we won't attempt to capture here.  For the most part though, `!FinalizerQueue -allReady` tells you objects that should be processed by the finalizer queue in the near future.

As you can see, our `BlockedFinalizer` objects are all in this list, along with a few other objects that happened to be finalizable but no longer referenced which should be cleaned up:

In [4]:
!FinalizeQueue -allReady -stat

0:000> !FinalizeQueue -allReady -stat
Calculating live objects, this may take a while...

Calculating live objects complete: 9,901 objects from 1,078 roots
SyncBlocks to be cleaned up: 0
Free-Threaded Interfaces to be released: 0
MTA Interfaces to be released: 0
STA Interfaces to be released: 0
----------------------------------

Heap 0
generation 0 has 16 objects (18e7206e410->18e7206e490)
generation 1 has 0 objects (18e7206e410->18e7206e410)
generation 2 has 0 objects (18e7206e410->18e7206e410)
Ready for finalization 1,011 objects (18e7206e590->18e72070528)
------------------------------
Statistics for all finalizable objects that are no longer rooted:
Statistics:
          MT Count TotalSize Class Name
7ff816d35e88     1        40 System.Gen2GcCallback
7ff816c06970     2        64 Internal.Win32.SafeHandles.SafeRegistryHandle
7ff816c7f5e8    12       864 System.Reflection.Emit.DynamicResolver
7ff816ccdeb8   999    23,976 BlockedFinalizer.BlockedFinalizer
Total 1,014 objects, 24,944 

Caching GC roots, this may take a while.
Subsequent runs of this command will be faster.


## Root causing 'excess memory'

Having too many objects 'ready' for finalization can lead to excess memory in your program.  The most common symptom here is increased managed or native memory, depending on what objects are building up that can't make it through the finalizer queue.

Before we dive further into debugging the issue, it's important to stop here and take stock of this state.  In our contrived example, we have blocked the finalizer queue and in the next sections we will explore how to see that.  However, in many real world scenarios we don't have to have a fully *blocked* finalizer thread in order to see this scenario.  The most common way this issue manifests itself is "slow" finalizers.

For example, imagine you create 25 finalizable objects per minute which eventually reach the finalizer queue (instead of being disposed of with the `IDisposable` interface/pattern).  Let's say that each of those objects takes 50 milliseconds to do some expensive cleanup operation.  This means that the finalizer thread can clean up ~20 of those objects per second.  Your process will be generating objects faster than they can be cleaned up.  The finalizer thread isn't *blocked* in this scenario, it eventually cleans up each individual object.  What's worse, you might have randomly paused the process to take a crash dump when a *different* object was being processed by the finalizer, which can be misleading.

When root causing this kind of issue (whether it's a blocked or 'slow' finalizer), it's important to simply take a look at the number of objects 'ready' for finalization and see if that number is increasing over time.  This typically means taking multiple crash dumps of the same process as your memory continues to grow.  Even if you don't have a series of crash dumps to compare, knowing what objects are ready for finalization and checking if that number seems abnormally large is a good idea to check.

## Finding the finalizer thread

While it can sometimes be misleading, it's a good idea to check and see what the finalizer thread is doing.  For that, we'll simply use the `!clrthreads` command in SOS, which gives us the managed view of threads in the process.  This command will not list out any thread where we have never run managed code before.  The `DBG` column tells us what the debugger calls that thread.  The `ID` column is the managed thread id (not useful to us in this case), and the `OSID` is what the operating system calls this thread.  Certain special threads, such as the finalizer, have a marker at the end to tell us what it is.  In this case we are looking for the `(Finalizer)` marker.  Below, you will see that thread 6 (OSID e6d8) is our finalizer thread.

In [5]:
!clrthreads

0:000> !clrthreads
ThreadCount:      8
UnstartedThread:  0
BackgroundThread: 6
PendingThread:    0
DeadThread:       0
Hosted Runtime:   no
                                                                                                            Lock  
 DBG   ID     OSID ThreadOBJ           State GC Mode     GC Alloc Context                  Domain           Count Apt Exception
   0    1     d574 0000018E70720630    2a020 Preemptive  0000018E74C3ED60:0000018E74C40938 0000018E7075E060 -00001 MTA 
   6    2     e6d8 0000018E707CBC40  2021220 Preemptive  0000018E74C35A28:0000018E74C36898 0000018E7075E060 -00001 Ukn (Finalizer) 
   7    3     d104 0000018E7072CD00    2b220 Preemptive  0000000000000000:0000000000000000 0000018E7075E060 -00001 MTA 
   8    4     6c50 0000018E00417760  202b020 Preemptive  0000000000000000:0000000000000000 0000018E7075E060 -00001 MTA 
   9    5     3e38 0000018E00464D40  202b220 Preemptive  0000018E74C36EC8:0000018E74C388B8 0000018E7075E060 -00001 MTA 
  10 

## Inspecting the finalizer thread

Now, we can inspect thread 6's callstack.  We swap to thread 6 with `~6s` (or use the OSID with `~~[e6d8]s`).  Then we can use the `k` command in WinDbg to list the callstack.  Here we use `kc20` where `c` gives us a clean (printable) output and `20` limits our frame count to 0x20.  From the output below you can see several things:

First, we can see that the finalizer thread is "awake" and actively processing objects because `coreclr!FinalizerThread::FinalizeAllObjects` is on the stack.  If the finalizer is "asleep", waiting for work to do we would see the `coreclr!FinalizerThread::WaitForFinalizerEvent` frame instead.  Second, it is processing our `BlockedFinalizer` object, and `BlockedFinalizer` is attempting to take a lock with `coreclr!JIT_MonReliableEnter_Portable` and `coreclr!Object::EnterObjMonitor`.  Third, we can see that we are blocking the thread because we are in a wait call `coreclr!CLREventBase::Wait` and similar frames.

If you are using SOS from within `dotnet-dump analyze` instead of WinDbg, you could also get this information from the managed portion of the stack with the `!clrstack` SOS command.  This would show you just the managed frames (and some .Net Runtime stack markers that are used to track state).  In this case it would show `BlockedFinalizer.Finalize` and that we are entering a monitor, but not the native frames that explicitly tell us whether the finalizer queue is sleeping or awake.  However, you can mostly infer that from whether any object has its Finalize method on the stack.

In [6]:
* Swap to thread 6 and show the stack.
* We can also use the ~~[e6d8]s syntax to switch to the thread using it's OS id.
~6s
kc20

0:000> ~6s
ntdll!NtWaitForMultipleObjects+0x14:
00007ff9`5a350af4 c3              ret
0:006> kc20
Call Site
ntdll!NtWaitForMultipleObjects
KERNELBASE!WaitForMultipleObjectsEx
coreclr!Thread::DoAppropriateAptStateWait
coreclr!Thread::DoAppropriateWaitWorker
coreclr!Thread::DoAppropriateWait
coreclr!CLREventBase::WaitEx
coreclr!CLREventBase::Wait
coreclr!AwareLock::EnterEpilogHelper
coreclr!AwareLock::EnterEpilog
coreclr!AwareLock::Enter
coreclr!SyncBlock::EnterMonitor
coreclr!ObjHeader::EnterObjMonitor
coreclr!Object::EnterObjMonitor
coreclr!JIT_MonEnter_Helper
coreclr!JIT_MonReliableEnter_Portable
BlockedFinalizer!BlockedFinalizer.BlockedFinalizer.Finalize
coreclr!FastCallFinalizeWorker
coreclr!MethodTable::CallFinalizer
coreclr!CallFinalizer
coreclr!FinalizerThread::FinalizeAllObjects
coreclr!FinalizerThread::FinalizerThreadWorker
coreclr!ManagedThreadBase_DispatchInner
coreclr!ManagedThreadBase_DispatchMiddle
coreclr!ManagedThreadBase_DispatchOuter
coreclr!ManagedThreadBase_NoADTrans

## Who owns the lock?

So we now know the finalizer thread is blocked by trying to enter a `Monitor` (this is the underpinning of `lock(obj)`).  We can also use SOS to find what thread is holding the lock with the `!syncblk` command.

.Net Runtime implementation details:  We can lock almost any object in .Net, and that information has to be held somewhere.  We keep a table of extra data that can be optionally associated with objects called the "SyncBlock Table".  Only objects which *need* extra data associated with them get a SyncBlock table entry allocated to them.  When you use the `lock` keyword (or `Monitor` class), we typically do not allocate a SyncBlock to the object if there is no contention.  Each object has an "object header", and part of that header is used for a low overhead spin-lock if there is no contention.  When two or more threads attempt to lock the same object, this spin-lock may be upgraded into a full lock, which is one of the things we track with the `SyncBlock` table.

By default, the `!syncblk` command only shows locks which have been upgraded to full SyncBlock table entries that are currently held by a thread.  You can use `!syncblk -all` to see the full table.  As you can see below, the lock is held by thread `6c50`, which is correct.  You can see the callstack of that thread by swapping over to that thread with `~~[6c50]s` followed by `k`.  Or you can execute a command on a thread with the `e` postfix.  In this case, we'll just look at the managed stack on that thread with `!clrstack` by using `~~[6c50]e !clrstack`, showing the managed stack of thread 6c50.

In [8]:
!syncblk
~~[6c50]e !clrstack

0:006> !syncblk
Index         SyncBlock MonitorHeld Recursion Owning Thread Info          SyncBlock Owner
    2 0000018E00462F00            3         1 0000018E00417760 6c50   8   0000018e74447c58 System.Object
-----------------------------
Total           2
CCW             0
RCW             0
ComClassFactory 0
Free            0
0:006> ~~[6c50]e !clrstack
OS Thread Id: 0x6c50 (8)
        Child SP               IP Call Site
000000370857F568 00007ff95a350624 [HelperMethodFrame: 000000370857f568] System.Threading.Thread.SleepInternal(Int32)
000000370857F660 00007FF875B8E671 System.Threading.Thread.Sleep(Int32)
000000370857F690 00007FF816B03858 BlockedFinalizer.Program.ThreadProc() [d:\git\debugging-dotnet\BlockedFinalizer\BlockedFinalizer.cs @ 85]
000000370857F970 00007ff87665b9e3 [DebuggerU2MCatchHandlerFrame: 000000370857f970] 


## Investigating this programmatically

You can write code using the [ClrMD library](https://github.com/Microsoft/clrmd) to automatically detect a small subset of this scenario.  ClrMD can only inspect the managed state of the process (it cannot inspect native stack frames or variables), but specifically for the case where a `Monitor` (`lock` keyword) is causing the issue, we can translate those SOS commands into C# code.

Note that this is an example of the power and flexibility of being able to use SOS-like commands from a C# program.  However, we have already investigated and solved the blocked finalizer repro we set out to using WinDbg.  We do not need to use ClrMD to investigate these issues.

### Getting started with ClrMD

We start by loading up ClrMD with the `Microsoft.Diagnostics.Runtime` package, and define a few variables.  Outside of a notebook, we should use the `using` keyword to properly dispose of `DataTarget` and `ClrRuntime`.

In [2]:
#r "nuget:Microsoft.Diagnostics.Runtime"
using Microsoft.Diagnostics.Runtime;

/* using */ DataTarget dt = DataTarget.LoadDump(@"..\BlockedFinalizer\bin\release\net8.0\BlockedFinalizer.dmp");
/* using */ ClrRuntime runtime = dt.ClrVersions[0].CreateRuntime();
ClrHeap heap = runtime.Heap;

Installed Packages Microsoft.Diagnostics.Runtime, 3.1.512801

### Find the finalizer thread's stack trace

We now need to find the finalizer thread and take a look at its managed stacktrace.  This is similar to using `!clrthreads` to find the finalizer, then `!clrstack` to see its managed stack trace.

In [4]:
// Print the stack of the finalizer thread.

ClrThread finalizerThread = runtime.Threads.Where(t => t.IsFinalizer).Single();
Console.WriteLine("Finalizer thread OSID 0x{0:x}, stack:", finalizerThread.OSThreadId);

foreach (ClrStackFrame frame in finalizerThread.EnumerateStackTrace())
    Console.WriteLine($"    {frame}");

Finalizer thread OSID 0xe6d8, stack:
    [HelperMethodFrame_1OBJ] (System.Threading.Monitor.ReliableEnter)
    BlockedFinalizer.BlockedFinalizer.Finalize()
    [DebuggerU2MCatchHandlerFrame]


### Are there any Monitor objects on the finalizer thread?

If there are any stack frames from the Monitor class `Monitor`, we should see if there is a stack root containing the object that we are attempting to `lock`.  To do this we will inspect the stack roots and what frame the root comes from.

You may notice that we do not simply look for arguments and local variables using ClrMD.  ClrMD itself is not a debugging API (that would be `ICorDebug`).  ClrMD is a ".Net runtime inspection" API.  The distinction matters here because the .Net runtime itself doesn't directly track local variable and argument addresses.  At least, it doesn't track it in a direct way that we can get through ClrMD.  Instead the runtime tracks roots which point to live objects.  In this case, that's sometimes enough to find the object we are looking for.

In [10]:
HashSet<ClrObject> objectsOfInterest = [];
foreach (ClrStackRoot root in finalizerThread.EnumerateStackRoots())
{
    if (root.StackFrame.Method?.Type?.Name != "System.Threading.Monitor")
        continue;
        
    objectsOfInterest.Add(root.Object);
}

Console.WriteLine($"Found {objectsOfInterest.Count} object{(objectsOfInterest.Count == 1 ? "" : "s")} of interest.");

Found 1 object of interest.


### Find out which thread is holding these Monitors via the SyncBlock table

Our heuristic might have found the wrong object, or the object in question might not have a SyncBlock created for it.  Similar to `!syncblk -all`, we will next loop through all SyncBlocks in the process and see if our objects of interest are on the SyncBlock table.  If so, we'll save those `SyncBlock` objects.

In [11]:

List<SyncBlock> syncBlocksOfInterest = [];
if (objectsOfInterest.Count > 0)
{
    Console.WriteLine("Found potential monitor objects:");
    foreach (ClrObject obj in objectsOfInterest)
        Console.WriteLine($"    {obj.Address:x12} {obj.Type?.Name}");

    Console.WriteLine();

    foreach (SyncBlock syncBlock in heap.EnumerateSyncBlocks())
    {
        // SyncBlock holds only the object's address.  We need to convert it into a ClrObject to use it here.
        ClrObject obj = heap.GetObject(syncBlock.Object);
        if (objectsOfInterest.Contains(obj))
            syncBlocksOfInterest.Add(syncBlock);
    }
}

Found potential monitor objects:
    018e74447c58 System.Object



### Finding the thread holding the lock

We now have a list of SyncBlocks matching objects rooted by `Monitor` frames.  We'll now go through each SyncBlock and see if the monitor is held, and if so by which thread.

In [13]:
List<(ClrObject Object, ClrThread Thread)> blockedObjects = [];
if (syncBlocksOfInterest.Count > 0)
{
    foreach (SyncBlock syncBlock in syncBlocksOfInterest)
    {
        if (syncBlock.IsMonitorHeld && syncBlock.HoldingThreadAddress != 0)
        {
            ClrThread holdingThread = runtime.Threads.FirstOrDefault(t => t.Address == syncBlock.HoldingThreadAddress);
            if (holdingThread != null)
                blockedObjects.Add((heap.GetObject(syncBlock.Object), holdingThread));
        }
    }
}

if (blockedObjects.Count > 0)
{
    Console.WriteLine("Found the following blocked Monitors on the Finalizer thread:");
    foreach ((ClrObject Object, ClrThread Thread) in blockedObjects)
        Console.WriteLine($"    {Object.Address:x12} {Object.Type?.Name} blocked by thread 0x{Thread.OSThreadId:x}");
}
else
{
    Console.WriteLine("Did not find a blocked Monitor object on the Finalizer thread.");
}


Found the following blocked Monitors on the Finalizer thread:
    018e74447c58 System.Object blocked by thread 0x6c50


## Wrap-up

One of the first steps in investigating higher than expected memory usage is checking if the Finalizer queue is blocked, and if there's more than expected finalizable objects.  This is because finalizers are one of the ways that native resources are released and given back to the operating system.

You can quickly check whether the finalizer queue is blocked by finding the finalizer thread with `!clrthreads` then using your debugger to see the native (or managed) stack trace of that thread.  You can see if there's an abnormally large amount of finalizable objects using `!fq -stat`, and whether there's an abnormally large amount of finalizable objects that the GC has already decided are unreferenced and ready for finalization using `!fq -allReady -stat`.

This works better when you have a baseline to compare against.  If you are investigating a memory leak, usually that means taking one crash dump early in the process's lifetime (before memory grows abnormally large) and one after memory has grown.  Otherwise, you will need to have a pretty good idea about the "normal" state of finalizable objects in your application, which is an intuition you can build by using the `!fq` command regularly.